# <font color="red">Réseau de neurones multicouches </font> à Feedforward
---
Les réseaux neuronaux sont nés du fait que tout ne peut pas être approximé par une régression linéaire. Il peut y avoir des formes potentiellement complexes dans les données qui ne peuvent être approximées que par des fonctions complexes, comme c'est le cas pour notre ensemble de données du code ASCII. Plus la fonction est complexe, meilleure est la précision des prédictions. 

Un réseau multicouches à feedforward consiste en un ensemble de neurones qui sont logiquement disposés en deux ou plusieurs couches. Il existe une couche d'entrée et une couche de sortie, chacune contenant au moins un neurone. Les neurones de la couche d'entrée sont hypothétiques dans la mesure où ils n'ont pas eux-mêmes d'entrées et où ils ne traitent rien, leur activation (sortie) est définie par 
l'entrée du réseau. Il y a généralement une ou plusieurs couches "cachées" entre les couches d'entrée et de sortie.

Le terme "feedforward" signifie que l'information circule dans un seul sens.  L'entrée dans les neurones de chaque couche provient exclusivement des sorties des neurones des couches précédentes, et les sorties de ces neurones passent exclusivement aux neurones des couches suivantes, la sortie de chaque neurone du réseau est fonction de l'entrée de ce neurone.

![1](images/neuron.png)

Les réseaux neuronaux sont composés de simples blocs de construction appelés neurones. Un neurone est une fonction mathématique qui prend des données comme entrée, effectue une transformation sur celles-ci et produit une sortie. Cela signifie que les neurones peuvent représenter n'importe quelle fonction mathématique ; cependant, dans les réseaux neuronaux, nous utilisons généralement des fonctions non linéaires.

En regardant le neurone ci-dessus, vous pouvez voir qu'il est composé de deux parties principales : la sommation et la fonction d'activation. Un neurone prend des données (x₁, x₂, x₃) comme entrée, multiplie chacune d'entre elles par un poids spécifique (w₁, w₂, w₃), puis transmet le résultat à une fonction non linéaire appelée fonction d'activation pour produire une sortie (y).

Dans ce tutoriel, la tâche consiste à créer un réseau de neurones pour classer les caractères du code ASCII à partir de ses bits. Nous avons un ensemble de entraînement, pour entraîner le classificateur et un ensemble de test, pour tester le résultat de l'entraînement.

---
## <font color="red">1 | LES </font>DONNÉES
---
La première chose que nous allons faire est d'importer les bibliothèques nécessaires et de charger notre ensemble de données. Les librairies que nous utiliserons sont :
* [numpy](https://numpy.org/) : Extension destinée à manipuler des matrices ou tableaux multidimensionnels ainsi que des fonctions mathématiques opérant sur ces tableaux.
* [pickle](https://docs.python.org/3/library/pickle.html) : Ce module permet de sauvegarder dans un fichier, au format binaire,  n'importe quel objet Python.
* [tabulate](https://pypi.org/project/tabulate/) : Pour afficher des tableaux d'une manière facile à lire. 
* [mathplotlib](https://matplotlib.org/) : C'est une bibliothèque pour tracer des graphiques en Python.

Pour une explication plus détaillée de l'ensemble des données, veuillez vous référer au notebook correspondant.

In [ ]:
import pickle
from tabulate import tabulate
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
with open("ascii_features.pickle", "rb") as db:
    features = pickle.load(db)
    
with open("ascii_outputs.pickle", "rb") as db:
    outputs = pickle.load(db)

X_train, X_test = features['TRAIN'], features['TEST']
outputs_train, outputs_test, classes = outputs['TRAIN'], outputs['TEST'], outputs['CLASSES']

labels_train = [string_label for label in outputs_train for string_label in label.keys()]
y_train = np.array([binary_label for label in outputs_train for binary_label in label.values()])

labels_test = [string_label for label in outputs_test for string_label in label.keys()]
y_test = np.array([binary_label for label in outputs_test for binary_label in label.values()])

---
## <font color="red">2 | LE </font>MODÈLE
---
Maintenant que vous avez chargé et préparé l'ensemble des données, commençons à construire le réseau de neurones pour faire des prédictions. Pour que les choses restent relativement simples, vous allez concevoir et coder un réseau de neurones à une couche cachée. Vous trouverez ci-dessous un aperçu de l'architecture :

![2](images/nn_arch.png)

La première couche est appelée couche d'entrée, et le nombre de nœuds dépendra du nombre de caractéristiques présentes dans votre ensemble de données. Dans notre cas, il y aura 8 nœuds car nous avons 8 primitives (bits).

La dernière couche du réseau de neurones est appelée la couche de sortie, et son nombre dépend de ce que vous essayez de prédire. Pour les tâches de régression et de classification binaire, vous pouvez utiliser un seul nœud, tandis que pour les problèmes multi-classes (comme le notre), vous utiliserez plusieurs nœuds, en fonction du nombre de classes.

Les couches situées entre la couche d'entrée et la couche de sortie sont celles où la magie opère, on les appelle les couches cachées. Les couches cachées peuvent être aussi profondes ou larges que vous le souhaitez, et si un réseau plus profond est préférable, le temps de calcul augmente également à mesure que vous vous enfoncez.

Le réseau neuronal ci-dessus aura une couche cachée et une couche de sortie. La couche d'entrée aura 8 nœuds car nous avons 8 primitives. La couche cachée peut accepter un nombre quelconque de nœuds, mais vous commencerez avec 6, et la couche de sortie, qui fait les prédictions, aura 5 nœuds.

### <font color="red">2.1 - Implémentation</font>
---
Le réseau neuronal artificiel est un algorithme d'apprentissage supervisé qui exploite un mélange de multiples hyperparamètres qui permettent d'approcher une relation complexe entre l'entrée et la sortie. 

Voici quelques-uns des hyperparamètres du réseau neuronal artificiel :
- Nombre de couches cachées
- Nombre d'unités cachées
- Fonction d'activation
- Taux d'apprentissage

### **Poids et biais**

Nous allons construire notre modèle à l'intérieur d'une classe appelée `NeuralNetwork`. Le constructeur prend des paramètres qui seront utilisés pour initialiser les poids et le biais, tels que le taux d'apprentissage, le nombre de neurones dans la couche d'entrée, le nombre de neurones dans la couche cachée et le nombre de neurones dans la couche de sortie.

Les poids sont des valeurs qui contrôlent la force de la connexion entre deux neurones. En d'autres termes, les entrées sont généralement multipliées par des poids, et cela définit l'influence de l'entrée sur la sortie. Les termes de biais sont des constantes supplémentaires attachées aux neurones et ajoutées à l'entrée pondérée avant que la fonction d'activation ne soit appliquée. Les termes de biais aident les modèles à représenter des modèles qui ne passent pas nécessairement par l'origine. 

Dans le bloc de code ci-dessous, vous allez créer la classe `NeuralNetwork` et initialiser les poids et biais :

In [ ]:
class NeuralNetwork():
    def __init__(self, nb_input_nodes, nb_hidden_nodes, nb_output_nodes, learning_rate):
        #  >>> HYPERPARAMETERS <<<
        self.nb_input_nodes = nb_input_nodes
        self.nb_output_nodes = nb_output_nodes
        self.nb_hidden_nodes = nb_hidden_nodes
        self.learning_rate = learning_rate

        # >>> WEIGHTS + BIASES <<<
        # Weight matrix from input to hidden layer
        self.W1 = np.random.randn(self.nb_input_nodes, self.nb_hidden_nodes)
        # Bias from input to hidden layer
        self.b1 = np.ones((self.nb_hidden_nodes))

        # Weight matrix from hidden to output layer
        self.W2 = np.random.randn(self.nb_hidden_nodes, self.nb_output_nodes)
        # Bias from hidden to output layer
        self.b2 = np.ones((self.nb_output_nodes))

Vous remarquerez qu'il y a deux tableaux de poids et de biais. Le premier tableau de poids (W1) aura des dimensions de 8 par 6, parce que vous avez 8 primitives d'entrée et 6 nœuds cachés, tandis que le premier biais (b1) sera un vecteur de taille 6 parce que vous avez 6 nœuds cachés.

Le deuxième tableau de pondération (W2) sera un tableau de 6 par 5 dimensions parce que vous avez 6 nœuds cachés et 5 nœuds de sortie, et enfin, le deuxième biais (b2) sera un vecteur de taille 5 parce que vous avez 5 sorties.

### **Function d'activation**

Maintenant que vous avez initialisé les poids et les biais, définissons la fonction d'activation. Une fonction d'activation est ce qui rend un réseau de neurones capable d'apprendre des fonctions non linéaires complexes. Les fonctions non linéaires sont difficiles à apprendre pour les algorithmes d'apprentissage machine traditionnels comme la logistique et la régression linéaire. La fonction d'activation est ce qui rend un réseau de neurones capable de comprendre ces fonctions.

La fonction d'activation est calculée par chaque nœud dans les couches cachées d'un réseau neuronal.Cela signifie que vous devrez faire passer les sommes pondérées par la fonction d'activation.

Il existe de nombreux types de fonctions d'activation utilisées dans les réseaux neuronaux, la plus populaire étant le sigmoïde. L'une des raisons d'utiliser la fonction sigmoïde est due à ses propriétés mathématiques, dans notre cas, à ses dérivées. Lorsque le réseau de neurones fait le backpropagation pour apprendre et mettre à jour les poids, nous utiliserons sa dérivée. À continuation on implémente la fonction `sigmoid` et `derivative_sigmoid`

\begin{equation}
    \sigma(x) = \frac{1}{1+\exp^{-x}} \\
    \sigma'(x) = \sigma(x) \cdot (1-\sigma(x))
\end{equation}


In [ ]:
class NeuralNetwork():
    def __init__(self, nb_input_nodes, nb_hidden_nodes, nb_output_nodes, learning_rate):
        #  >>> HYPERPARAMETERS <<<
        self.nb_input_nodes = nb_input_nodes
        self.nb_output_nodes = nb_output_nodes
        self.nb_hidden_nodes = nb_hidden_nodes
        self.learning_rate = learning_rate

        # >>> WEIGHTS + BIASES <<<
        # Weight matrix from input to hidden layer
        self.W1 = np.random.randn(self.nb_input_nodes, self.nb_hidden_nodes)
        # Bias from input to hidden layer
        self.b1 = np.ones((self.nb_hidden_nodes))

        # Weight matrix from hidden to output layer
        self.W2 = np.random.randn(self.nb_hidden_nodes, self.nb_output_nodes)
        # Bias from hidden to output layer
        self.b2 = np.ones((self.nb_output_nodes))

    def sigmoid(self, z):
        s = 1 / (1 + np.exp(-z))
        return s

    def derivative_sigmoid(self, z):
        return self.sigmoid(z) * (1 - self.sigmoid(z))

### **Propagation en avant**

La propagation en avant est le nom donné à la série de calculs effectués par le réseau de neurones avant qu'une prédiction ne soit faite. Le processus de feed-forward est assez simple. La formule $z = (weights \cdot input) + b$ calcule $z$ qui est passé dans les couches, qui contient des fonctions d'activation spécifiques. Ces fonctions d'activation produisent la sortie $f(z)$. La sortie de la couche actuel sera l'entrée de la couche suivant et ainsi de suite.

Dans votre réseau à deux couches, vous effectuerez le calcul suivant pour la propagation directe :

* Calculez la somme pondérée entre les poids de l'entrée et ceux de la première couche, puis ajoutez le biais : **Z2 = (W1 * X) + b**
* Faites passer le résultat par la fonction d'activation sigmoïde : **A2 = sigmoïde(Z2)**
* Calculez la somme pondérée entre la sortie (A2) de l'étape précédente et les poids de la deuxième couche - ajoutez également le biais : **Z3 = (W2 * A2) + b2**
* Calculer la fonction de sortie en faisant passer le résultat par une fonction sigmoïde : **A3 = sigmoïde(Z3)**
* Et enfin, calculer la perte entre le résultat prévu et les étiquettes réelles : **perte(A3, Y)**

À continuation, l'implémentation de la fonction `forward` d'accord à la formule précédente : 

In [ ]:
class NeuralNetwork():
    def __init__(self, nb_input_nodes, nb_hidden_nodes, nb_output_nodes, learning_rate):
        #  >>> HYPERPARAMETERS <<<
        self.nb_input_nodes = nb_input_nodes
        self.nb_output_nodes = nb_output_nodes
        self.nb_hidden_nodes = nb_hidden_nodes
        self.learning_rate = learning_rate

        # >>> WEIGHTS + BIASES <<<
        # Weight matrix from input to hidden layer
        self.W1 = np.random.randn(self.nb_input_nodes, self.nb_hidden_nodes)
        # Bias from input to hidden layer
        self.b1 = np.ones((self.nb_hidden_nodes))

        # Weight matrix from hidden to output layer
        self.W2 = np.random.randn(self.nb_hidden_nodes, self.nb_output_nodes)
        # Bias from hidden to output layer
        self.b2 = np.ones((self.nb_output_nodes))

    def sigmoid(self, z):
        s = 1 / (1 + np.exp(-z))
        return s

    def derivative_sigmoid(self, z):
        return self.sigmoid(z) * (1 - self.sigmoid(z))

    def forward(self, X):
        # Forward propagation through our network
        self.A1 = X
        self.Z2 = np.dot(self.A1, self.W1) + self.b1
        self.A2 = self.sigmoid(self.Z2)
        self.Z3 = np.dot(self.A2, self.W2) + self.b2
        self.A3 = self.sigmoid(self.Z3)
        return self.A3

### **Mesurer l'erreur**

La fonction de perte est un moyen de mesurer la performance de votre réseau par rapport aux valeurs réelles.

La fonction de perte la plus couramment utilisée est l'erreur quadratique moyenne. Comme nous traitons d'un problème de classification multi-classes, le résultat sera une distribution de probabilité. nous devons la comparer à nos valeurs réelles, qui sont aussi une distribution de probabilité, et trouver l'erreur.

Nous allons utiliser la fonction de Cross-Entropy pour évaluer l’erreur. Cette fonction mesure les performances d'un modèle de classification dont le résultat est une probabilité.


\begin{equation}
    Cross Entropy = -\sum_{i=1}^{C} y_{i}\log(\hat{y}_i)
\end{equation}

Ou :

* $C$ : C'est le nombre de classes
* $y$ : Étiquettes de classes pour chaque exemple
* $\hat{y}_i$ : C'est la probabilité prédite que l'entrée est de la classe $c$

À conitnuation, on fait l'implémentation en python de la fonction `entropy_loss` :

In [ ]:
class NeuralNetwork():
    def __init__(self, nb_input_nodes, nb_hidden_nodes, nb_output_nodes, learning_rate):
        #  >>> HYPERPARAMETERS <<<
        self.nb_input_nodes = nb_input_nodes
        self.nb_output_nodes = nb_output_nodes
        self.nb_hidden_nodes = nb_hidden_nodes
        self.learning_rate = learning_rate

        # >>> WEIGHTS + BIASES <<<
        # Weight matrix from input to hidden layer
        self.W1 = np.random.randn(self.nb_input_nodes, self.nb_hidden_nodes)
        # Bias from input to hidden layer
        self.b1 = np.ones((self.nb_hidden_nodes))

        # Weight matrix from hidden to output layer
        self.W2 = np.random.randn(self.nb_hidden_nodes, self.nb_output_nodes)
        # Bias from hidden to output layer
        self.b2 = np.ones((self.nb_output_nodes))

    def sigmoid(self, z):
        s = 1 / (1 + np.exp(-z))
        return s

    def derivative_sigmoid(self, z):
        return self.sigmoid(z) * (1 - self.sigmoid(z))

    def entropy_loss(self, y, y_pred):
        # Prevent taking the log of 0
        eps = np.finfo(float).eps
        return -np.sum(y * np.log(y_pred + eps))

    def forward(self, X):
        # Forward propagation through our network
        self.A1 = X
        self.Z2 = np.dot(self.A1, self.W1) + self.b1
        self.A2 = self.sigmoid(self.Z2)
        self.Z3 = np.dot(self.A2, self.W2) + self.b2
        self.A3 = self.sigmoid(self.Z3)
        return self.A3

### **Propagation en arrière**

Fondamentalement, `backward` calcule l'erreur à partir de la sortie de `forward` et de la valeur réelle. Cette erreur est rétro-propagée à toutes les matrices de pondération en calculant les gradients de chaque couche et ces pondérations sont mises à jour. Regardons donc les méthodes de `backward` et `train`:

Pour chaque itérations, nous appliquons l'algorithme back-prop, évaluons l'erreur et le gradient par rapport aux poids. Nous utilisons ensuite le taux d’apprentissage et les gradients pour mettre à jour les poids.

Veuillez vous référer aux notes de cours pour l'explication complète des dérivés utilisés dans l'algorithme de back-prop.

In [ ]:
class NeuralNetwork():
    def __init__(self, nb_input_nodes, nb_hidden_nodes, nb_output_nodes, learning_rate):
        #  >>> HYPERPARAMETERS <<<
        self.nb_input_nodes = nb_input_nodes
        self.nb_output_nodes = nb_output_nodes
        self.nb_hidden_nodes = nb_hidden_nodes
        self.learning_rate = learning_rate
        self.loss = []

        # >>> WEIGHTS + BIASES <<<
        # Weight matrix from input to hidden layer
        self.W1 = np.random.randn(self.nb_input_nodes, self.nb_hidden_nodes)
        # Bias from input to hidden layer
        self.b1 = np.ones((self.nb_hidden_nodes))

        # Weight matrix from hidden to output layer
        self.W2 = np.random.randn(self.nb_hidden_nodes, self.nb_output_nodes)
        # Bias from hidden to output layer
        self.b2 = np.ones((self.nb_output_nodes))

    def sigmoid(self, z):
        s = 1 / (1 + np.exp(-z))
        return s

    def derivative_sigmoid(self, z):
        return self.sigmoid(z) * (1 - self.sigmoid(z))

    def entropy_loss(self, y, y_pred):
        # Prevent taking the log of 0
        eps = np.finfo(float).eps
        return -np.sum(y * np.log(y_pred + eps))

    def forward(self, X):
        # Forward propagation through our network
        self.A1 = X
        self.Z2 = np.dot(self.A1, self.W1) + self.b1
        self.A2 = self.sigmoid(self.Z2)
        self.Z3 = np.dot(self.A2, self.W2) + self.b2
        self.A3 = self.sigmoid(self.Z3)
        return self.A3

    def backward(self, X, y):
        m = X_train.shape[0]  # number of examples

        # Error in output
        #d3
        dZ3 = self.A3-y
        # Delta for the weights w2
        dW2 = (1./m) * np.dot(self.A2.T, dZ3)
        # Delta for the bias b2
        db2 = np.sum(dZ3, axis=0)  # sum across columns

        # d2
        dA2 = np.dot(dZ3, self.W2.T)
        dZ2 = dA2 * self.derivative_sigmoid(self.Z2)
        # Delta for the weights w1
        dW1 = (1./m) * np.dot(X.T, dZ2)
        # Delta for the bias b1
        db1 = (1./m) * np.sum(dZ2, axis=0)  # sum across columns

        # Wights and biases update
        self.W2 = self.W2 - self.learning_rate * dW2
        self.b2 = self.b2 - self.learning_rate * db2
        self.W1 = self.W1 - self.learning_rate * dW1
        self.b1 = self.b1 - self.learning_rate * db1

    def train (self, X, y, nb_iterations):
        for i in range(nb_iterations):
            y_pred = self.forward(X)
            loss = self.entropy_loss(y, y_pred)
            self.loss.append(loss)
            self.backward(X, y)

            if i == 0 or i == nb_iterations-1:
                print(f"Iteration: {i+1}")
                print(tabulate(zip(X, y, [np.round(y_pred) for y_pred in self.A3] ), headers=["Input", "Actual", "Predicted"]))
                print(f"Loss: {loss}")                
                print("\n")

    def predict(self, X):
        return np.round(self.forward(X))

Dans l'ensemble d'entraînement on a 102 exemples et 8 primitives par exemple.

In [ ]:
X_train.shape

On a aussi les étiquettes binaires pour les 102 exemples.

In [ ]:
y_train.shape

Il est temps de créer notre modèle et de le former, en utilisant 1000 itérations :

In [ ]:
NN = NeuralNetwork(
    nb_input_nodes = X_train.shape[1],  # 8 features
    nb_hidden_nodes = 6,  # Number of neurons in the hidden layer
    nb_output_nodes = len(classes),  # 5 classes we can use y_train.shape[1] too
    learning_rate = 0.1
)

Les résultats de la première et de la dernière itération sont présentés comme une sortie de l'entraînement. Vous pouvez voir comment le classement s'améliore de l'itération 1, où pratiquement toutes les prédictions sont incorrectes, à 1000 où presque toutes sont correctes.

In [ ]:
nb_iterations = 10000
NN.train(X_train, y_train, nb_iterations)

Nous voyons ci-dessous comment la fonction de perte se comporte à chaque itération :

In [ ]:
plt.plot(NN.loss)
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.title("Loss curve for training")
plt.show()

Enfin, nous pouvons faire des prédictions avec le modèle entraîné en utilisant la fonction `predict` qui fait un passage en avant de l'ensemble des données de test en utilisant les poids et les biais déjà ajustés pendant l'entraînement.

In [ ]:
y_pred = NN.predict(X_test)
print(tabulate(zip(X_test, labels_test, [classes.get(tuple(o), "--") for o in y_pred]), headers=["Input", "Actual", "Predicted"]))